In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from sentence_transformers import SentenceTransformer
import pickle
import os

# Web Based Loader

In [15]:
# # web based loader
# from langchain_community.document_loaders import WebBaseLoader
# import bs4

# ## load,chunk and index the content of the html page

# loader=WebBaseLoader(web_paths=("insert URL",),
#                      bs_kwargs=dict(parse_only=bs4.SoupStrainer(
#                          class_=("post-title","post-content","post-header")

#                      )))

# text_documents=loader.load()

In [16]:
## Pdf reader 
loader = PyPDFLoader('pdfs/subhas.pdf')  # Works on Windows & Linux
docs = loader.load()


In [17]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents= text_splitter.split_documents(docs) 
documents[:3]

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-01-24T07:16:15+00:00', 'author': 'Jaya Surya', 'moddate': '2025-01-24T07:16:16+00:00', 'source': 'pdfs/subhas.pdf', 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content='Subhas Chandra Bose[h] (23 January 1897 – 18 August 1945) was an Indian nationalist whose \ndefiance of British authority in India made him a hero among many Indians, but his wartime alliances \nwith Nazi Germany and Imperial Japan left a legacy vexed by authoritarianism, anti-Semitism, \nand military failure. The honorific \'Netaji\' (Hindustani: "Respected Leader") was first applied to Bose \nin Germany in early 1942—by the Indian soldiers of the Indische Legion and by the German and \nIndian officials in the Special Bureau for India in Berlin. It is now used throughout India.[i] \nBose was born into wealth and privilege in a large Bengali family in Orissa during the British Raj. The \nearly recipien

In [18]:
# Function to save FAISS index as a pickle file
def save_faiss_to_pickle(faiss_index, file_path="faiss_index.pkl"):
    with open(file_path, "wb") as f:
        pickle.dump(faiss_index, f)
    print(f"FAISS index saved as a pickle file: {file_path}")

# Function to load FAISS index from a pickle file
def load_faiss_from_pickle(file_path="faiss_index.pkl"):
    with open(file_path, "rb") as f:
        faiss_index = pickle.load(f)
    print("FAISS index loaded successfully from pickle file.")
    return faiss_index

# Function to create FAISS index and save as a pickle file
def create_faiss_index_and_save(documents, file_path="faiss_index.pkl"):
    embeddings = HuggingFaceEmbeddings()
    db = FAISS.from_documents(documents, embeddings)
    save_faiss_to_pickle(db, file_path)
    print("FAISS index created and saved successfully!")

# Load documents from a PDF file
loader = PyPDFLoader("pdfs/subhas.pdf")
text_documents = loader.load()

# Create and save the FAISS index
create_faiss_index_and_save(text_documents, "faiss_index.pkl")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_5192\921366152.py:16: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


FAISS index saved as a pickle file: faiss_index.pkl
FAISS index created and saved successfully!


In [19]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Use the BAAI model for embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Create a FAISS vector database
db = FAISS.from_documents(documents, embeddings)

In [20]:
query= "where did subhas born"

result=db.similarity_search(query)
result[0].page_content

"Subhas imbibed a nurturing spirit, looking for situations in which to help people in distress, \npreferring gardening around the house to joining in sports with other boys.[8] His father, who was \nreserved in manner and busy with professional life, was a distant presence in a large family, causing \nSubhas to feel he had a nondescript childhood.[57] Still, Janakinath read English literature avidly—\nJohn Milton, William Cowper, Matthew Arnold, and Shakespeare's Hamlet being among his \nfavourites; several of his sons were to become English literature enthusiasts like him. \nIn 1909, the 12-year-old Subhas Bose followed his five brothers to the Ravenshaw Collegiate \nSchool in Cuttack.[8] Here, Bengali and Sanskrit were also taught, as were ideas from Hindu scriptures \nsuch as the Vedas and the Upanishads not usually picked up at home.[8] Although his western \neducation continued apace, he began to wear Indian clothes and engage in religious speculation. To"

In [21]:
## Load Ollama LAMA2 LLM model
llm=Ollama(model="Llama2")

## ChatPrompt Template

prompt = ChatPromptTemplate.from_template("""
Answer the following question based **only** on the provided context.  
If the context **does not contain relevant information**, respond with **"I don’t have enough information to answer this."**  

Provide a **concise and factual** response, avoiding unnecessary details.  

<context>  
{context}  
</context>  
Question: {input}  
""")



In [22]:
# Chain
document_chain=create_stuff_documents_chain(llm,prompt)
#Retriever
retriever=db.as_retriever()

retrieval_chain= create_retrieval_chain(retriever,document_chain)

In [23]:
response=retrieval_chain.invoke({"input":"who is subhas"})
response['answer']

'Based on the provided context, Subhas Chandra Bose is a person who was born in 1893 in India. He had a nurturing spirit and preferred gardening to joining sports with other boys. His father was distant and busy with professional life, causing Subhas to feel he had a nondescript childhood. Subhas followed his five brothers to the Ravenshaw Collegiate School in Cuttack, where he studied Bengali and Sanskrit, as well as ideas from Hindu scriptures such as the Vedas and the Upanishads. He also wore Indian clothes and engaged in religious speculation. In 1916, Subhas was alleged to have masterminded or participated in an incident involving a professor at Presidency College, Calcutta. Despite his preoccupation with religion and culture, Subhas was able to focus on his studies and secure the second position in the matriculation examination conducted under the auspices of the University of Calcutta in 1912. He later studied philosophy at Presidency College and was awarded the Subhas Chandra B

In [24]:
response=retrieval_chain.invoke({"input":"what color hat did he wear"})
response['answer']

"I don't have enough information to answer this question based on the provided context. The context does not provide any information about Bose's hat color."

In [25]:
response=retrieval_chain.invoke({"input":"What was his work in Germany?"})
response['answer']

'Based on the provided context, it can be inferred that Subhas Chandra Bose\'s work in Germany involved collaborating with the Nazi leadership to promote Indian independence. However, there is limited information available in the context to provide a detailed answer to this question. Therefore, I must respond with "I don\'t have enough information to answer this."'

In [26]:
response=retrieval_chain.invoke({"input":"What he did in Japan?"})
response['answer']



"Based on the provided context, it is known that Subhas Chandra Bose traveled to Japan in 1943 after being disillusioned with Germany's inability to help India gain independence. Once in Japan, he met with Japanese leaders and established a Provisional Government of Free India, which was recognized by nine Axis states. The INA, or Indian National Army, was also formed under Bose's leadership, comprising Indian prisoners of war captured by the Japanese in Singapore. The INA participated in operations behind enemy lines in Eastern India, including the Japanese thrust towards Imphal and Kohima."

In [ ]:
# Store conversation history
chat_history = []

def chat_with_ai(user_input):
    """
    Function to handle chat interactions with the AI model.
    It keeps track of the chat history and retrieves relevant context.
    """
    global chat_history  # Keep track of previous messages
    
    # Format history as a single string (keeping last 5 exchanges)
    history_text = "\n".join(chat_history[-5:])
    
    # Query the retrieval chain
    response = retrieval_chain.invoke({
        "history": history_text,
        "input": user_input
    })
    
    # Get the assistant's response
    ai_response = response['answer']
    
    # Update chat history
    chat_history.append(f"User: {user_input}")
    chat_history.append(f"Assistant: {ai_response}")
    
    return ai_response

# Run the chat system interactively
while True:
    user_input = input("\nYou: ")  # Get user input
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Chat session ended.")
        break
    response = chat_with_ai(user_input)
    print(f"\nUser: {user_input}")
    print(f"Assistant: {response}")



User: who is subhas
Assistant: Based on the provided context, Subhas Chandra Bose is a person who was born in 1893 in India. He had a nondescript childhood and was interested in helping people in distress, preferring gardening to joining sports with other boys. He attended Ravenshaw Collegiate School in Cuttack, where he learned Bengali and Sanskrit, as well as ideas from Hindu scriptures such as the Vedas and the Upanishads. Bose was also interested in Western education and wore Indian clothes while engaging in religious speculation. He was known to have a strong focus on his studies, competing, and succeeding in exams. In 1912, he secured the second position in the matriculation examination conducted under the auspices of the University of Calcutta. Bose followed his five brothers again in 1913 to Presidency College, Calcutta, where he chose to study philosophy.

User: when did he go to japan
Assistant: Based on the provided context, it is possible that Subhas Chandra Bose traveled 